In [3]:
import d3rlpy
from d3rlpy.algos import COMBO
from sklearn.model_selection import train_test_split
import gymnasium as gym
import numpy as np
import encoders

## Params

In [5]:
seed = 1
d3rlpy.seed(seed)
use_gpu = True
# prepare environment
env = gym.make("InvertedPendulum-v4")
eval_env = gym.make("InvertedPendulum-v4")
env.reset(seed=seed)
eval_env.reset(seed=seed)

(array([ 0.00023643,  0.00900927, -0.00711681,  0.00897299]), {})

## Create dataset

In [14]:
actor_encoder = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# setup algorithm
sac = d3rlpy.algos.SAC(
    batch_size=256,
    actor_encoder_factory=actor_encoder,
    actor_learning_rate=3e-4,
    critic_learning_rate=3e-4,
    temp_learning_rate=3e-4,
    use_gpu=use_gpu
)

# prepare utilities
buffer = d3rlpy.online.buffers.ReplayBuffer(maxlen=1000000, env=env)

# start training
sac.fit_online(
    env,
    buffer,
    eval_env=eval_env,
    n_steps=100000,
    n_steps_per_epoch=1000,
    update_interval=1,
    update_start_step=1000,
    tensorboard_dir='tensorboard_logs'
)

2023-10-01 15:22:21 [info     ] Directory is created at d3rlpy_logs/SAC_online_20231001152221
2023-10-01 15:22:21 [debug    ] Building model...
2023-10-01 15:22:21 [debug    ] Model has been built.
2023-10-01 15:22:21 [info     ] Parameters are saved to d3rlpy_logs/SAC_online_20231001152221/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0003, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma': 0.99, 'generated_maxlen': 100000, 'initial_temperature': 1.0, 'n_critics': 2, '

  0%|          | 0/100000 [00:00<?, ?it/s]

2023-10-01 15:22:23 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_1000.pt
2023-10-01 15:22:23 [info     ] SAC_online_20231001152221: epoch=1 step=1000 epoch=1 metrics={'time_inference': 0.0011901423931121827, 'time_environment_step': 6.751775741577148e-05, 'time_step': 0.0013019063472747804, 'rollout_return': 10.06060606060606, 'evaluation': 14.7} step=1000
2023-10-01 15:22:37 [info     ] Model parameters are saved to d3rlpy_logs/SAC_online_20231001152221/model_2000.pt
2023-10-01 15:22:37 [info     ] SAC_online_20231001152221: epoch=2 step=2000 epoch=2 metrics={'time_inference': 0.0012940735816955567, 'time_environment_step': 0.0001036219596862793, 'time_sample_batch': 0.000309492826461792, 'time_algorithm_update': 0.011871881484985351, 'temp_loss': 1.372351692557335, 'temp': 0.8709251883625985, 'critic_loss': 0.6754881671965122, 'actor_loss': -4.277053299486637, 'time_step': 0.013659589290618897, 'rollout_return': 17.75, 'evaluation': 102.4} ste

In [15]:
# export replay buffer as MDPDataset
dataset = buffer.to_mdp_dataset()

# save MDPDataset
dataset.dump('d3rlpy_data/inverted_pendulum2.h5')

## Load the dataset

In [6]:
dataset = d3rlpy.dataset.MDPDataset.load('d3rlpy_data/inverted_pendulum2.h5')

In [7]:
train_episodes, test_episodes = train_test_split(dataset, random_state=seed)

## Dynamics learning

In [17]:
# encoder_factory = encoders.InvertedPendulumEncoderFactory()
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics(learning_rate=1e-4, use_gpu=True, encoder_factory="inverted_pendulum")

Using InvertedPendulumEncoderFactory


In [18]:
# same as algorithms
dynamics.fit(train_episodes,
             eval_episodes=test_episodes,
             n_epochs=100,
             scorers={
                'observation_error': d3rlpy.metrics.scorer.dynamics_observation_prediction_error_scorer,
                'reward_error': d3rlpy.metrics.scorer.dynamics_reward_prediction_error_scorer,
                'variance': d3rlpy.metrics.scorer.dynamics_prediction_variance_scorer,
             },
            tensorboard_dir='tensorboard_logs/dynamics')

2023-10-02 23:06:32 [debug    ] RoundIterator is selected.
2023-10-02 23:06:32 [info     ] Directory is created at d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632
2023-10-02 23:06:32 [debug    ] Building models...
2023-10-02 23:06:32 [debug    ] Models have been built.
2023-10-02 23:06:32 [info     ] Parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632/params.json params={'action_scaler': None, 'batch_size': 100, 'discrete_action': False, 'encoder_factory': {'type': 'inverted_pendulum', 'params': {'hidden_units': [256, 256], 'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None, 'use_dense': False}}, 'gamma': 1.0, 'generated_maxlen': 100000, 'learning_rate': 0.0001, 'n_ensembles': 5, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0.0001, 'amsgrad': False}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'use_gpu': 0, 'variance_type': 'max', 'algorithm':

Epoch 1/100:   0%|          | 0/751 [00:00<?, ?it/s]

2023-10-02 23:06:43 [info     ] ProbabilisticEnsembleDynamics_20231002230632: epoch=1 step=751 epoch=1 metrics={'time_sample_batch': 0.00022852055718514637, 'time_algorithm_update': 0.013406936719160423, 'loss': -21.364473361799742, 'time_step': 0.013703783406081117, 'observation_error': 0.05243828512231935, 'reward_error': 0.0017379385478908265, 'variance': 0.06518403561763936} step=751
2023-10-02 23:06:43 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632/model_751.pt


Epoch 2/100:   0%|          | 0/751 [00:00<?, ?it/s]

2023-10-02 23:06:55 [info     ] ProbabilisticEnsembleDynamics_20231002230632: epoch=2 step=1502 epoch=2 metrics={'time_sample_batch': 0.00022441315428712238, 'time_algorithm_update': 0.01318661001487356, 'loss': -31.360013313204565, 'time_step': 0.013474410129450609, 'observation_error': 0.030806009419821413, 'reward_error': 0.0008161839346589291, 'variance': 0.02939949919303506} step=1502
2023-10-02 23:06:55 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632/model_1502.pt


...

2023-10-02 23:14:45 [info     ] ProbabilisticEnsembleDynamics_20231002230632: epoch=41 step=30791 epoch=41 metrics={'time_sample_batch': 0.00025066840505790454, 'time_algorithm_update': 0.013568961033014738, 'loss': -75.9969762631961, 'time_step': 0.013890091176039369, 'observation_error': 0.02967821012231883, 'reward_error': 2.973742936621061e-06, 'variance': 0.027824332189573146} step=30791
2023-10-02 23:14:45 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632/model_30791.pt


Epoch 42/100:   0%|          | 0/751 [00:00<?, ?it/s]

2023-10-02 23:14:57 [info     ] ProbabilisticEnsembleDynamics_20231002230632: epoch=42 step=31542 epoch=42 metrics={'time_sample_batch': 0.00026208741687109243, 'time_algorithm_update': 0.013898081849322973, 'loss': -76.14587828259017, 'time_step': 0.0142314999144817, 'observation_error': 0.035268183058480035, 'reward_error': 1.1854234170584247e-05, 'variance': 0.027841275058009225} step=31542
2023-10-02 23:14:57 [info     ] Model parameters are saved to d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632/model_31542.pt


Epoch 43/100:   0%|          | 0/751 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Load Dynamics

In [19]:
# load trained dynamics model
dynamics_model_path = "d3rlpy_logs/ProbabilisticEnsembleDynamics_20231002230632"
dynamics = d3rlpy.dynamics.ProbabilisticEnsembleDynamics.from_json(dynamics_model_path + '/params.json')
dynamics.load_model(dynamics_model_path + '/model_31542.pt')

Using InvertedPendulumEncoderFactory
2023-10-02 23:15:47 [warning  ] Unused arguments are passed.   algorithm=ProbabilisticEnsembleDynamics gamma=1.0 generated_maxlen=100000 n_steps=1 real_ratio=1.0


## Train Offline RL Algorithm

In [20]:
encoders = d3rlpy.models.encoders.DefaultEncoderFactory(dropout_rate=0.2)
# give COMBO as the generator argument.
combo = COMBO(dynamics=dynamics, critic_encoder_factory=encoders, actor_encoder_factory=encoders,
              use_gpu=use_gpu)

In [21]:
combo.fit(dataset = train_episodes, eval_episodes=test_episodes, n_steps=100000, n_steps_per_epoch=1000, tensorboard_dir="tensorboard_logs",
         scorers={
            'environment': d3rlpy.metrics.scorer.evaluate_on_environment(eval_env)
        })

2023-10-02 23:16:08 [debug    ] RandomIterator is selected.
2023-10-02 23:16:08 [info     ] Directory is created at d3rlpy_logs/COMBO_20231002231608
2023-10-02 23:16:08 [debug    ] Building models...
2023-10-02 23:16:08 [debug    ] Models have been built.
2023-10-02 23:16:08 [info     ] Parameters are saved to d3rlpy_logs/COMBO_20231002231608/params.json params={'action_scaler': None, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'actor_learning_rate': 0.0001, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'batch_size': 256, 'conservative_weight': 1.0, 'critic_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': 0.2}}, 'critic_learning_rate': 0.0003, 'critic_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'gamma':

Epoch 1/100:   0%|          | 0/1000 [00:00<?, ?it/s]

2023-10-02 23:16:21 [debug    ] 250000 transitions are generated. fake_transitions=250000 real_transitions=75144


KeyboardInterrupt: 

## Load the Policy

In [9]:
trained_policy = COMBO()
# initialize with dataset
trained_policy.build_with_dataset(dataset)
# Load entire model parameters.
trained_policy.load_model('d3rlpy_logs/COMBO_20230929153035/model_53000.pt')

## See the policy running

In [10]:
scorer = d3rlpy.metrics.scorer.evaluate_on_environment(eval_env, render=True)
mean_episode_return = scorer(trained_policy)